* will add distribution of zmag_close and zmag_open (distribution over all beads, all cycles) -> look at datatable to show quantiles
* finish implementing fixed beads
* rewrite ramp.py library

In [1]:
import data
%run ramp_module
import bokeh.plotting
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import *
from bokeh.layouts import column, row
import time
output_notebook()


ramp called as main


Loading BokehJS ...

In [2]:
# plotting functions
def plot_dzdt(bcids, rpdata, **kwargs):
    count = 0
    palette = kwargs.get("palette",bokeh.models.palettes.Dark2[8])
    add_map = kwargs.get("add_map",rpdata.dzdt==rpdata.dzdt)
    collapse = kwargs.get("collapse", False)
    detected = kwargs.get("detected",False)
    x_name = kwargs.get("x_name","t")
    circles = kwargs.get("circles",False)
    fig = bokeh.plotting.figure(width=400,height=400)
    for bcid in bcids:
        color = palette[count%len(palette)]
        bid, cid = bcid
        x_range=rpdata.dataz[x_name,cid][add_map[bcid]]
        if collapse : 
            x_range=rpdata.dataz[x_name,bcids[0][1]][add_map[bcids[0]]]
        if circles:
            fig.circle(x=x_range,y=rpdata.dzdt[bcid][add_map[bcid]], line_color = color)
        else:
            fig.line(x=x_range,y=rpdata.dzdt[bcid][add_map[bcid]], line_color = color)
        if detected and not collapse:
            fig.circle(x=rpdata.dataz[x_name,cid][rpdata.det[bcid]],y=rpdata.dzdt[bcid][rpdata.det[bcid]], line_color = color)
        if detected and collapse:
            fig.circle(x=rpdata.dataz[x_name,bcids[0][1]][rpdata.det[bcid]],y=rpdata.dzdt[bcid][rpdata.det[bcid]], line_color = color)

        count+=1
    return fig

def plot_dataz(bcids, rpdata, **kwargs):
    count = 0
    palette = kwargs.get("palette",bokeh.models.palettes.Dark2[8])
    add_map = kwargs.get("add_map",rpdata.dataz==rpdata.dataz)
    collapse = kwargs.get("collapse", False)
    circles = kwargs.get("circles",False)
    x_name = kwargs.get("x_name","t")
    detected = kwargs.get("detected",False)
    fig = bokeh.plotting.figure(width=400,height=400)
    for bcid in bcids:
        color = palette[count%len(palette)]
        bid,cid = bcid
        x_range=rpdata.dataz[x_name,cid][add_map[bcid]]
        if collapse : 
            x_range=rpdata.dataz[x_name,bcids[0][1]][add_map[bcids[0]]]
        if circles:
            fig.circle(x=x_range,y=rpdata.dataz[bcid][add_map[bcid]], line_color = color)
        else:
            fig.line(x=x_range,y=rpdata.dataz[bcid][add_map[bcid]], line_color = color)
        if detected and not collapse:
            fig.circle(x=rpdata.dataz[x_name,cid][rpdata.det[bcid]],y=rpdata.dataz[bcid][rpdata.det[bcid]], line_color = color)
        if detected and collapse:
            fig.circle(x=rpdata.dataz[x_name,bcids[0][1]][rpdata.det[bcid]],y=rpdata.dataz[bcid][rpdata.det[bcid]], line_color = color)

        count+=1
    return fig

In [3]:
# Please use this cell to give me necessary information 
#track_file="/media/data/helicon/remi/2016-11-08/ramp_5HPs_mix.trk" # test on time scale change not adding anything to this data
#track_file="/media/data/helicon/zhen/2016-12-02/ramp_mirB_3233_120_F1_M2A.trk"
#track_file="/media/data/helicon/sylwia/2016-08-26_CTAG_5_HPs_mix/ramps/ramp_5HPs_mix.trk"

# bead 2, 6, 14 of the following ramp shows some structural blocking
#track_file="/media/data/helicon/sylwia/2016-08-26_AGAT_5_HPs_mix/ramps/ramp_5HPs_mix.trk" 
track_file="/home/david/Desktop/RAMP_5HPmix.trk" 

In [4]:
track=data.Track(path=track_file)
mod=RampModel()
rpdata=RampData(model = mod)
rpdata.setTrack(track)

tmp = time.time()
keep = rpdata.areGoodBeadCycle()
print(time.time()-tmp)

rpdata.setTrack(track)
tmp = time.time()
rpdata.clean()
print(time.time()-tmp)



rpfulldata = RampData(model = mod)
rpfulldata.setTrack(track)

0.0959932804107666
0.6628415584564209


In [5]:
times = [i for i in rpdata.dataz.keys() if i[0]=="t"] 
zmags = [i for i in rpdata.dataz.keys() if i[0]=="zmag"]
print("zmags=",zmags)
fig = bokeh.plotting.figure(width=600, height=600)
fig.circle(rpdata.dataz[times[1]],rpdata.dataz[zmags[1]])
print("est=",(rpdata.dataz[zmags[1]]==-0.168).sum())
unit = rpdata._estimateUnitScale()
print("unit=",unit)
show(fig)


zmags= [('zmag', 15), ('zmag', 1), ('zmag', 8), ('zmag', 16), ('zmag', 2), ('zmag', 5), ('zmag', 12), ('zmag', 6), ('zmag', 9), ('zmag', 17), ('zmag', 3), ('zmag', 10), ('zmag', 13), ('zmag', 18), ('zmag', 7), ('zmag', 14), ('zmag', 0), ('zmag', 11), ('zmag', 4)]
est= 0
unit= 35.0


In [9]:
# test on fixed bead ids
t=rpdata.estExtHPsize()
t = rpdata.dataz[rpdata.bcids].max() -rpdata.dataz[rpdata.bcids].min()
print(t)
#clids ={i[0] for i in rpdata.bcids if all(closed[i])}
t2=rpdata.getFixedBeadIds()

(41, 13)    1.706645
(31, 6)     1.489261
(89, 11)    1.619938
(92, 16)    0.262532
(43, 3)     1.570261
(91, 13)    1.521951
(46, 12)    1.585976
(20, 7)     1.207505
(94, 10)    1.579666
(45, 9)     1.506879
(93, 15)    0.719739
(32, 2)     1.860579
(80, 12)    1.604451
(35, 15)    1.538068
(83, 9)     1.571118
(34, 8)     1.458917
(82, 14)    1.602160
(37, 13)    1.538885
(24, 14)    1.552001
(14, 1)     1.451481
(52, 17)    1.574401
(48, 17)    1.598328
(28, 10)    1.596171
(31, 15)    1.496402
(89, 12)    1.619416
(79, 1)     1.525072
(30, 16)    1.458729
(17, 13)    1.598650
(91, 2)     1.503221
(65, 11)    1.555685
              ...   
(63, 2)     1.581475
(24, 0)     1.521683
(1, 2)      1.559811
(75, 7)     1.623207
(49, 8)     1.518722
(4, 9)      1.597859
(52, 3)     1.571547
(77, 5)     1.540854
(54, 9)     1.555162
(41, 4)     1.714562
(89, 2)     1.612422
(66, 4)     1.863151
(43, 10)    1.570542
(56, 18)    1.619308
(46, 5)     1.582814
(94, 3)     1.583752
(33, 16)    1

In [7]:
# prints the sections of the data which is tagged as a possible structural event
se = rpdata.can_be_structure_event()
bcids=[(10,i) for i in range(rpdata.ncycles)]
#bcids = list(rpdata.bcids)
#bcids+=[(25,i) for i in range(rpdata.ncycles)]
print(rpdata.beads())
print(rpdata.ncycles)
hover = HoverTool(tooltips =[("(x,y)","(@x,@y)")])
fig1 = plot_dataz(bcids,rpdata,x_name="zmag",collapse=True, detected =True)
fig1.add_tools(hover)
#fig2 = plot_dataz(bcids,rpdata,add_map=se,collapse=True,circles=True)
fig2 = plot_dzdt(bcids,rpdata,x_name="t",collapse=True, detected =True)
#fig2 = plot_dataz(bcids,rpdata,add_map=se,x_name="zmag",collapse=True,circles=True)

show(row(fig1,fig2))

{1, 2, 4, 7, 8, 10, 11, 12, 14, 17, 18, 19, 20, 22, 23, 24, 25, 28, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 65, 66, 67, 70, 71, 75, 77, 79, 80, 81, 82, 83, 88, 89, 91, 92, 93, 94}
19


In [8]:
# testing clusterization algorithm (dbscan)
bcids=[(14,i) for i in range(rpdata.ncycles)]
se = rpdata.can_be_structure_event()
se_z = rpdata.dataz[se]
zvalues={b:[] for b in rpdata.beads()}
for bcid in rpdata.bcids:
    zvalues[bcid[0]].extend(se_z[bcid].dropna().values)
print(zvalues)
clusterids = pd.DataFrame({k:pd.Series(cluster1D_dbscan(numpy.array(v))) for k, v in zvalues.items()})
    

{1: [0.33112562, 0.32521746, 0.31825089, 0.32276577, 0.31637529, 0.31415135, 0.31259727, 0.29161721, 0.18943664, 0.19155341, 0.19226347, 0.20655829, 0.85817212, 0.85095102, 0.84662372, 0.84197491, 0.83077484, 0.82690305, 0.81221968, 0.80880338, 0.80619091, 0.79548657, 0.79839379, 0.78807789, 0.77520317, 0.77518976], 2: [0.77963763, 0.084054142, 0.77027297, 0.20578125, 0.2041066, 0.20295444, 0.19385773, 1.3983747, 1.4007326, 1.4072704, 1.4109681, 1.4116915, 1.4114906, 1.4098562, 1.4119997, 1.4113567, 1.4124418, 1.4111691, 1.4144915, 1.4140093, 1.4134465, 1.413594, 1.4122409, 1.4136877, 1.4153758, 1.4155633, 1.4167022, 1.4128839, 1.4141567, 1.4161662, 1.4148132, 1.4142907, 1.4159787, 1.4146792, 1.4157912, 1.414947, 1.4160188, 1.4171845, 1.415416, 1.414237, 1.4172782, 1.416193, 1.4160858, 1.4146255, 1.4144915, 1.4119864, 1.4083557, 1.4033049, 1.4026886, 1.4015633, 1.3916895, 1.3886484, 1.3849508, 1.3775153, 1.3755325, 1.3689946, 1.3638366, 1.363274, 1.3609027, 1.3541104, 1.3470232, 1.3395

ValueError: Metric '{}' not valid for algorithm 'auto'

In [ ]:
# investigate the size of the hysteresis

# size of hysteresis estimated from the difference between zmagop and zmagcl
hys_size = rpdata.zmagOpen()-rpdata.zmagClose()
# for each bead compute the size the hysteresis (boxplot over the cycles)
hys_size.describe()
fig = bokeh.plotting.figure(width=500,height=500)
fig.line(x=hys_size.index,y=hys_size.mean(axis=1))
show(fig)

In [ ]:
# investigate relationship between the size of the hysteresis and other measured quantities (zmag_op, size hairpin)
hys_size = rpdata.zmagOpen()-rpdata.zmagClose()
hp_size = rpdata.estHPsize()
print(hys_size.std(axis=0))
for b in rpdata.beads():
    if hys_size.mean(axis=1)[b]>0.22:
        print(b)
fig = bokeh.plotting.figure()
#fig.circle(x=hp_size.values.flatten(),y=hys_size.values.flatten(),size=10)
fig.circle(x=hp_size.mean(axis=1),y=hys_size.mean(axis=1),size=10)

fig.add_tools(HoverTool(tooltips = [("(x,y)","(@x,@y)")]))

show(fig)

In [ ]:
# investigate means to differentiate between structural blocking and oligo binding. 
# Need to compare the same hairpins between different experiments. 
# It seems like very few (if any) structural blocking are observable

se = rpdata.can_be_structure_event()
print("beads = ",rpdata.beads())
# have to look at each bead and find distribution of z[se]
bcids =[(14,i) for i in range(rpdata.ncycles)]

palette = bokeh.models.palettes.Dark2[8]
count = 0
figs = []
for bcid in bcids :
    count +=1
    zs=rpdata.dataz[se][bcid].dropna().values.tolist()
    hist,edges = numpy.histogram(zs)
    fig = bokeh.plotting.figure()
    fig.quad(top=hist,bottom = 0, left=edges[:-1],right =edges[1:],color = palette[count%len(palette)])
    figs.append(fig)

show(column(*figs))